## Deep Research

One of the classic cross-business Agentic use cases! This is huge.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">A Deep Research agent is broadly applicable to any business area, and to your own day-to-day activities. You can make use of this yourself!
            </span>
        </td>
    </tr>
</table>

In [1]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [2]:
load_dotenv(override=True)

True

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.

### Important note - API charge of WebSearchTool

This is costing me 2.5 cents per call for OpenAI WebSearchTool. That can add up to $2-$3 for the next 2 labs. We'll use free and low cost Search tools with other platforms, so feel free to skip running this if the cost is a concern. Also student Christian W. pointed out that OpenAI can sometimes charge for multiple searches for a single call, so it could sometimes cost more than 2.5 cents per call.

Costs are here: https://platform.openai.com/docs/pricing#web-search

In [ ]:
INSTRUCTIONS = "你是一名研究助理。給定一個搜尋詞，你需要在網路上搜尋該詞並產生一份簡明的摘要。摘要必須為 2-3 段且少於 300 字。抓住主要重點。寫得簡潔，無需完整句子或良好的文法。這將被用於綜合報告，因此抓住精髓並忽略任何冗長內容是至關重要的。除了摘要本身，不要包含任何額外的評論。"

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")], # 使用低上下文大小的網頁搜尋工具
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"), # 強制使用網頁搜尋工具
)


In [4]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

截至2025年，AI代理框架的發展迅速，以下是幾個領先的框架：

**LangChain**：一個強大的開源框架，旨在簡化大型語言模型（LLM）應用的開發。其模組化和可擴展的架構提供了豐富的工具，包括各種LLM的接口、提示模板、代理模組、記憶系統和動態檢索組件，適用於對話代理、文檔分析和代碼生成等多種應用。 ([dev.to](https://dev.to/surgedatalab/best-5-frameworks-for-agentic-ai-in-2025-enabling-next-gen-intelligent-multi-agent-systems-40ce?utm_source=openai))

**AutoGen**：由微軟開發，專注於協調多個AI代理，形成自主的事件驅動系統，能夠無縫處理複雜的多代理任務。其特點包括多代理協調、可擴展的運行時收斂，適用於需要事件驅動工作流的企業級應用，如客戶服務自動化。 ([linkedin.com](https://www.linkedin.com/pulse/ai-agent-frameworks-june-2025-comprehensive-overview-chadi-abi-fadel-wcu5c?utm_source=openai))

**CrewAI**：採用基於角色的代理協作方法，促進創建專門的代理，協同完成複雜項目，類似於團隊環境。其特點包括動態任務規劃、實時性能監控，以及協調各種代理作為獨立工作者（“船員”），適用於需要代理之間協作的複雜項目，如軟件開發或項目管理。 ([linkedin.com](https://www.linkedin.com/pulse/ai-agent-frameworks-june-2025-comprehensive-overview-chadi-abi-fadel-wcu5c?utm_source=openai))

此外，**Amazon Bedrock AgentCore**和**Eliza**等新興平台也在2025年引起關注。Amazon Bedrock AgentCore旨在簡化先進AI代理的開發和部署，提供可擴展的無伺服器部署、上下文管理和安全服務訪問等功能。Eliza則是一個開源的Web3友好型代理框架，旨在無縫集成Web3應用，實現區塊鏈數據的讀寫和智能合約交互。 ([techradar.com](https://www.techradar.com/pro/aws-looks-to-super-charge-ai-agents-with-amazon-bedrock-agentcore?utm_source=openai), [arxiv.org](https://arxiv.org/abs/2501.06781?utm_source=openai))

這些框架的出現和發展，標誌著AI代理技術在2025年的快速進步，為開發者提供了多樣化的工具和平台，以構建更智能、更高效的AI代理系統。 

### As always, take a look at the trace

https://platform.openai.com/traces

### We will now use Structured Outputs, and include a description of the fields

In [5]:
# 注意上方關於 WebSearchTool 成本的說明

HOW_MANY_SEARCHES = 3  # 定義要執行的搜尋次數

# 定義指令，指導代理如何生成搜尋計畫
INSTRUCTIONS = f"你是一個有幫助的研究助理。給定一個查詢，請提出一組網頁搜尋來最佳回答該查詢。輸出 {HOW_MANY_SEARCHES} 個搜尋詞。"

# 使用 Pydantic 定義回應的結構化輸出 Schema
# 感謝學生 Wes C. 發現並修復了一個棘手的錯誤！

class WebSearchItem(BaseModel):
    reason: str = Field(description="為什麼這個搜尋對查詢很重要的推理。")  # 搜尋理由
    query: str = Field(description="用於網頁搜尋的搜尋詞。")  # 搜尋詞

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="為最佳回答查詢而執行的一組網頁搜尋。")  # 搜尋計畫

# 定義一個名為 PlannerAgent 的代理，用於生成搜尋計畫
planner_agent = Agent(
    name="PlannerAgent",  # 代理名稱
    instructions=INSTRUCTIONS,  # 指令
    model="gpt-4o-mini",  # 使用的模型
    output_type=WebSearchPlan,  # 輸出的結構化類型
)


In [6]:

message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To gather information on the most recent AI agent frameworks being developed in 2025.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='To explore different platforms and tools that support AI agent development as of 2025.', query='AI agent development platforms 2025'), WebSearchItem(reason='To find emerging technologies and trends in AI agent frameworks and their applications in 2025.', query='emerging AI agent technologies 2025')]


In [35]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ed@edwarddonner.com") # Change this to your verified email
    to_email = To("ed.donner@gmail.com") # Change this to your email
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [ ]:
send_email

In [37]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)



In [7]:
# 定義指令，指導 WriterAgent 如何生成報告
INSTRUCTIONS = (
    "您是一位資深研究員，負責為研究查詢撰寫一份連貫的報告。"
    "您將獲得原始查詢以及研究助理完成的一些初步研究。\n"
    "您應該首先為報告制定一個大綱，描述報告的結構和流程。"
    "然後，生成報告並將其作為最終輸出。\n"
    "最終輸出應為 markdown 格式，且應該詳細且內容豐富。目標為 5-10 頁內容，至少 1000 字。"
)

# 定義報告數據的結構化輸出 Schema
class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")  # 簡短摘要，2-3 句描述研究結果
    markdown_report: str = Field(description="The final report")  # 最終報告內容
    follow_up_questions: list[str] = Field(description="Suggested topics to research further")  # 建議進一步研究的問題

# 定義一個名為 WriterAgent 的代理，用於生成詳細的研究報告
writer_agent = Agent(
    name="WriterAgent",  # 代理名稱
    instructions=INSTRUCTIONS,  # 指令
    model="gpt-4o-mini",  # 使用的模型
    output_type=ReportData,  # 輸出的結構化類型
)


### The next 3 functions will plan and execute the search, using planner_agent and search_agent

In [ ]:
async def plan_searches(query: str):
    """ 使用 planner_agent 計劃要執行的搜尋 """
    print("Planning searches...")  # 計劃搜尋
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")  # final_output = WebSearchPlan
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ 對搜尋計劃中的每個項目執行搜尋 """
    print("Searching...")  # 開始搜尋
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)  # 等待所有搜尋完成
    print("Finished searching")  # 搜尋完成
    return results

async def search(item: WebSearchItem):
    """ 使用 search_agent 執行每個搜尋項目 """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"  # 搜尋輸入內容
    result = await Runner.run(search_agent, input)  # 使用 WebSearchTool 執行搜尋
    return result.final_output  # 返回搜尋結果

### The next 2 functions write a report and email it

In [9]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

### Showtime!

In [ ]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    # await send_email(report)
    print(report)




Starting research...
Planning searches...
Will perform 3 searches
Searching...
Finished searching
Thinking about report...


### As always, take a look at the trace

https://platform.openai.com/traces

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thanks.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00cc00;">Congratulations on your progress, and a request</h2>
            <span style="color:#00cc00;">You've reached an important moment with the course; you've created a valuable Agent using one of the latest Agent frameworks. You've upskilled, and unlocked new commercial possibilities. Take a moment to celebrate your success!<br/><br/>Something I should ask you -- my editor would smack me if I didn't mention this. If you're able to rate the course on Udemy, I'd be seriously grateful: it's the most important way that Udemy decides whether to show the course to others and it makes a massive difference.<br/><br/>And another reminder to <a href="https://www.linkedin.com/in/eddonner/">connect with me on LinkedIn</a> if you wish! If you wanted to post about your progress on the course, please tag me and I'll weigh in to increase your exposure.
            </span>
        </td>
    </tr>